# Decorators

* Functions
* Wrap other functions and enhance their behavior
* Examples of higher order functions
* Have their own syntax, using "@" (syntactic sugar)

An example without the syntactic sugar.

Below, `wrapper()` is the higher order function that gets returned in `be_polite(fn)`…

In [23]:
def be_polite(fn):
    def wrapper():
        print("What a pleasure to meet you!")
        fn()
        print("Have a great day!")
    return wrapper

def greet():
    print("My name is Colt.")

def rage():
    print("I HATE YOU!!!")

This is where we decorate it, the syntactic sugar will normally be used instead of lines like below.

In [25]:
my_greet = be_polite(greet)

polite_rage = be_polite(rage)

In [26]:
my_greet()

What a pleasure to meet you!
My name is Colt.
Have a great day!


In [27]:
polite_rage()

What a pleasure to meet you!
I HATE YOU!!!
Have a great day!


The normal way to use a decorator is with the syntactic sugar "@" like so…

In [29]:
def be_polite(fn):
    def wrapper():
        print("What a pleasure to meet you!")
        fn()
        print("Have a great day!")
    return wrapper

@be_polite
def greet():
    print("My name is Matt.")

@be_polite
def rage():
    print("SHADDAP JERKY!!!")

### Explanation:

* You'll call `greet()` or `rage()`
* It'll pass the output from either of those functions as called into the decorator `be_polite(fn)` as the `fn` argument
* The output from `greet()` or `rage()` gets piped into the `wrapper()` higher order function inside `be_polite()`

In [30]:
greet()

What a pleasure to meet you!
My name is Matt.
Have a great day!


In [31]:
rage()

What a pleasure to meet you!
SHADDAP JERKY!!!
Have a great day!


## Decorators with Different Signatures

**NOTE:** `wrapper()` as the higher order function is just a standard. It can actually be any name as long as it gets returned in the end of the decorator function.

In [44]:
def shout(fn):
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs).upper()
    return wrapper

@shout
def greet(name):
    return f"Hi, I'm {name}."

@shout
def order(main, side):
    return f"Hi, I'd like the {main}, with a side of {side}, please."

@shout
def lol():
    return "lol"

One argument passed, we've seen it before…

In [45]:
greet("Todd")

"HI, I'M TODD."

Two arguments this time, the get passed into `*args` in `wrapper()`

In [46]:
order("burger", "fries")

"HI, I'D LIKE THE BURGER, WITH A SIDE OF FRIES, PLEASE."

Or even no arguments passed…

In [47]:
lol()

'LOL'

`**kwargs` will be activated if you pass named arguments…

In [48]:
order(side="mashed potatoes", main="fishwich")

"HI, I'D LIKE THE FISHWICH, WITH A SIDE OF MASHED POTATOES, PLEASE."

## Preserve Metadata

In [63]:
def log_function_data(fn):
    def wrapper(*args, **kwargs):
        """I AM A WRAPPER FUNCTION"""
        print(f"you are about to call: {fn.__name__}")
        print(f"here's the documentation: {fn.__doc__}")
        return fn(*args, **kwargs)
    return wrapper

@log_function_data
def add(x, y):
    """Adds two numbers together"""
    return x + y

In [64]:
print(add(10, 30))

you are about to call: add
here's the documentation: Adds two numbers together
40


But note what happens when call dunders on the add function…

In [65]:
print(add.__doc__)

I AM A WRAPPER FUNCTION


In [66]:
print(add.__name__)

wrapper


To remedy this, you'd use the `functools` package…

```python
from functools import wraps

def my_decorator(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        # do something here
        pass
    return wrapper
```

In [67]:
from functools import wraps

def log_function_data(fn):
    @wraps(fn) # <-- this is all you need to add
    def wrapper(*args, **kwargs):
        """I AM A WRAPPER FUNCTION"""
        print(f"you are about to call: {fn.__name__}")
        print(f"here's the documentation: {fn.__doc__}")
        return fn(*args, **kwargs)
    return wrapper

@log_function_data
def add(x, y):
    """Adds two numbers together"""
    return x + y

In [68]:
print(add.__doc__)

Adds two numbers together


In [69]:
print(add.__name__)

add


In [70]:
help(add)

Help on function add in module __main__:

add(x, y)
    Adds two numbers together

